<a href="https://colab.research.google.com/github/maxrusk/m/blob/main/Last_name%2C_First_name_cognitive_models_assignment_3_question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Question 2. Multigenerational Individual and Population Belief Updating**

## Below we define a model about the *individual's* updating cognitive beliefs based on observations using Bayesian inference.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def update_beliefs(prior_mean, prior_variance, observations, observation_variance):
      # Update beliefs based on observations using Bayesian inference.

      likelihood_mean = np.mean(observations)
      posterior_variance = 1 / (1 / prior_variance + len(observations) / observation_variance)
      posterior_mean = posterior_variance * (prior_mean / prior_variance + sum(observations) / observation_variance)
      return posterior_mean, posterior_variance

def simulate_individual_belief_evolution(true_theta, prior_mean, prior_variance, observation_variance, observations_per_generation, num_generations):
    generation_means = [prior_mean]
    generation_variances = [prior_variance]

    for g in range(num_generations):
        observations = np.random.normal(true_theta, np.sqrt(observation_variance), observations_per_generation)
        posterior_mean, posterior_variance = update_beliefs(generation_means[-1], generation_variances[-1], observations, observation_variance)
        generation_means.append(posterior_mean)
        generation_variances.append(posterior_variance)

        # Calculate 95% Confidence Interval for the current generation
        ci_lower = posterior_mean - 1.96 * np.sqrt(posterior_variance)
        ci_upper = posterior_mean + 1.96 * np.sqrt(posterior_variance)

        # Print out the estimated mean and CI for the current generation
        print(f"Generation {g+1}: Estimated Mean = {posterior_mean:.2f}, 95% CI = [{ci_lower:.2f}, {ci_upper:.2f}]")

    # Plotting the evolution of beliefs across generations
    plt.figure(figsize=(10, 6))
    plt.plot(range(num_generations + 1), generation_means, marker='o', label='Estimated Mean')
    plt.fill_between(range(num_generations + 1),
                     np.array(generation_means) - 1.96 * np.sqrt(generation_variances),
                     np.array(generation_means) + 1.96 * np.sqrt(generation_variances),
                     color='blue', alpha=0.2, label='95% Confidence Interval')
    plt.axhline(y=true_theta, color='r', linestyle='-', label='True State')
    plt.xlabel('Generation')
    plt.ylabel('Belief about θ')
    plt.title("Evolution of an Individual's Beliefs Across Generations")
    plt.legend()
    plt.grid(True)
    plt.show()


## **2a. The cell below defines a set of parameter values for running the model. Execute the cell three times (you may need to copy and paste it for each run), and then describe the meanings of the parameters and the resulting outputs in your own words.**

In [ ]:
# Example usage:
true_theta = 50
prior_mean = 40
prior_variance = 100
observation_variance = 25
observations_per_generation = 1
num_generations = 10

simulate_individual_belief_evolution(true_theta, prior_mean, prior_variance, observation_variance, observations_per_generation, num_generations)

## Now we are going to define a *population* model of non-social and social bilief updating across generations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def compare_learning_scenarios(true_theta, prior_mean, prior_variance, observation_variance,
                               num_generations, population_size, observations_per_generation, sample_size):
    def update_beliefs(prior_mean, prior_variance, observations, observation_variance):
        # Standard Bayesian update using private observations.
        likelihood_mean = np.mean(observations)
        posterior_variance = 1 / (1 / prior_variance + len(observations) / observation_variance)
        posterior_mean = posterior_variance * (prior_mean / prior_variance + sum(observations) / observation_variance)
        return posterior_mean, posterior_variance

    def update_beliefs_social(prior_mean, prior_variance, social_means, social_variances):
        # Bayesian update combining the agent’s own prior with social signals.
        # Here, each social signal is the previous generation’s posterior belief (mean and variance).
        prior_precision = 1.0 / prior_variance
        social_precision_sum = sum(1.0 / sv for sv in social_variances)
        total_precision = prior_precision + social_precision_sum
        posterior_variance = 1.0 / total_precision
        weighted_sum = prior_mean / prior_variance + sum(sm / sv for sm, sv in zip(social_means, social_variances))
        posterior_mean = posterior_variance * weighted_sum
        return posterior_mean, posterior_variance

    def simulate_generation(population_means, population_variances, true_theta, observation_variance, observations_per_individual):
        # Non-social learning: each individual gathers private observations.
        new_means = []
        new_variances = []
        for mean, variance in zip(population_means, population_variances):
            observations = np.random.normal(true_theta, np.sqrt(observation_variance), observations_per_individual)
            posterior_mean, posterior_variance = update_beliefs(mean, variance, observations, observation_variance)
            new_means.append(posterior_mean)
            new_variances.append(posterior_variance)
        return new_means, new_variances

    def simulate_generation_with_random_sampling(population_means, population_variances, true_theta, observation_variance, observations_per_individual, sample_size):
        # Social learning: each individual updates its belief by observing a random sample of peers
        # (i.e. their posterior means and variances) from the previous generation.
        new_means = []
        new_variances = []
        pop_size = len(population_means)
        for i in range(pop_size):
            sampled_indices = np.random.choice(pop_size, sample_size, replace=True)
            social_means = [population_means[idx] for idx in sampled_indices]
            social_variances = [population_variances[idx] for idx in sampled_indices]
            posterior_mean, posterior_variance = update_beliefs_social(population_means[i],
                                                                       population_variances[i],
                                                                       social_means,
                                                                       social_variances)
            new_means.append(posterior_mean)
            new_variances.append(posterior_variance)
        return new_means, new_variances

    def simulate_population_evolution(num_generations, population_size, true_theta, prior_mean, prior_variance,
                                      observation_variance, observations_per_individual, social=False, sample_size=20):
        # All individuals start with the same prior.
        population_means = np.array([prior_mean] * population_size, dtype=float)
        population_variances = np.array([prior_variance] * population_size, dtype=float)
        individual_means_over_time = [population_means.copy()]
        individual_variances_over_time = [population_variances.copy()]

        for g in range(num_generations):
            if social:
                if g == 0:
                    # Bootstrap: first update is private so that beliefs shift away from the prior.
                    population_means, population_variances = simulate_generation(
                        population_means, population_variances, true_theta, observation_variance, observations_per_individual)
                else:
                    # Pure social update: each individual observes the previous generation’s posteriors.
                    population_means, population_variances = simulate_generation_with_random_sampling(
                        population_means, population_variances, true_theta, observation_variance, observations_per_individual, sample_size)
            else:
                # Non-social: every generation gathers private observations.
                population_means, population_variances = simulate_generation(
                    population_means, population_variances, true_theta, observation_variance, observations_per_individual)

            individual_means_over_time.append(population_means.copy())
            individual_variances_over_time.append(population_variances.copy())

        return np.array(individual_means_over_time), np.array(individual_variances_over_time)

    # Run the simulations.
    non_social_means_over_time, non_social_variances_over_time = simulate_population_evolution(
        num_generations, population_size, true_theta, prior_mean, prior_variance, observation_variance,
        observations_per_generation, social=False)

    social_means_over_time, social_variances_over_time = simulate_population_evolution(
        num_generations, population_size, true_theta, prior_mean, prior_variance, observation_variance,
        observations_per_generation, social=True, sample_size=sample_size)

    # Compute average means and convergence error.
    average_non_social_means = np.mean(non_social_means_over_time, axis=1)
    average_social_means = np.mean(social_means_over_time, axis=1)

    error_non_social = np.abs(average_non_social_means - true_theta)
    error_social = np.abs(average_social_means - true_theta)

    # Plot individual trajectories.
    fig, axes = plt.subplots(1,2, figsize=(14,6))
    for i in range(population_size):
        axes[0].plot(range(num_generations+1), non_social_means_over_time[:, i], alpha=0.3, lw=1)
    axes[0].set_title('Non-Social Learning')
    axes[0].axhline(y=true_theta, color='r', linestyle='-', label='True θ')
    axes[0].set_xlabel('Generation')
    axes[0].set_ylabel('Belief about θ')

    for i in range(population_size):
        axes[1].plot(range(num_generations+1), social_means_over_time[:, i], alpha=0.3, lw=1)
    axes[1].set_title('Social Learning')
    axes[1].axhline(y=true_theta, color='r', linestyle='-', label='True θ')
    axes[1].set_xlabel('Generation')

    plt.legend()
    plt.show()

    # Plot convergence error over generations.
    plt.figure(figsize=(14,6))
    plt.plot(range(num_generations+1), error_non_social, marker='o', linestyle='--', color='darkorange', label='Non-Social Error')
    plt.plot(range(num_generations+1), error_social, marker='s', linestyle='-', color='purple', label='Social Error')
    plt.xlabel('Generation')
    plt.ylabel('Absolute Error |mean - true θ|')
    plt.title('Convergence Error Over Generations')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

    # Plot the means and 95% confidence intervals.
    ci_non_social = np.var(non_social_means_over_time, axis=1, ddof=1)
    ci_social = np.var(social_means_over_time, axis=1, ddof=1)

    plt.figure(figsize=(14,6))
    plt.plot(range(num_generations+1), average_non_social_means, marker='o', linestyle='--', color='darkorange', label='Non-Social Mean')
    plt.fill_between(range(num_generations+1), average_non_social_means - 2*np.sqrt(ci_non_social),
                     average_non_social_means + 2*np.sqrt(ci_non_social), color='orange', alpha=0.1, label='Non-Social 95% CI')
    plt.plot(range(num_generations+1), average_social_means, marker='s', linestyle='-', color='purple', label='Social Mean')
    plt.fill_between(range(num_generations+1), average_social_means - 2*np.sqrt(ci_social),
                     average_social_means + 2*np.sqrt(ci_social), color='violet', alpha=0.1, label='Social 95% CI')
    plt.axhline(y=true_theta, color='r', linestyle='-', label='True θ')
    plt.xlabel('Generation')
    plt.ylabel('Mean Belief and Confidence Interval')
    plt.title('Mean Beliefs and 95% CI Over Generations')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()

    # Print generation-by-generation results.
    for g in range(num_generations+1):
        ns_ci_lower = average_non_social_means[g] - 2*np.sqrt(ci_non_social[g])
        ns_ci_upper = average_non_social_means[g] + 2*np.sqrt(ci_non_social[g])
        s_ci_lower = average_social_means[g] - 2*np.sqrt(ci_social[g])
        s_ci_upper = average_social_means[g] + 2*np.sqrt(ci_social[g])
        print(f"Generation {g}:")
        print(f"  Non-Social: Mean = {average_non_social_means[g]:.2f}, Error = {error_non_social[g]:.2f}, 95% CI = [{ns_ci_lower:.2f}, {ns_ci_upper:.2f}]")
        print(f"  Social:     Mean = {average_social_means[g]:.2f}, Error = {error_social[g]:.2f}, 95% CI = [{s_ci_lower:.2f}, {s_ci_upper:.2f}]\n")



## **2b. Now, we run the model to compare multigenerational population dynamics under non-social and social learning. Execute the model below and describe the simulations in your own words, including the parameters, generated figures, *and* results.**

In [ ]:
# Copy and paste this cell to run the function with different sets of parameters
compare_learning_scenarios(
    true_theta=50,
    prior_mean=40,
    prior_variance=100,
    observation_variance=25,
    num_generations=10,
    population_size=100,
    observations_per_generation=1,   #for non-social learning
    sample_size=1                    #for social learning
)

## **2c. Exploring the Impact of Observations and Sample Size: The previous tests assumed observations_per_generation = 1 for non-social learning and sample_size = 1 for social learning. Now, adjust both parameters to 2, 3, 5, 10, and 20, respectively. Run the model for each parameter set (you may need to copy and paste the previous cell for each run), then observe and compare the results as these values increase. How do these changes impact the outcomes? Specifically, from a "socio-cognitive" perspective: (i) Do your experiments provide an good illustration that social learning require less effort and fewer cognitive resources compared to non-social learning, as claimed by Hardy et al. (2022)? (ii) Do your experiments provide an good illustration of "rational information accumulation" through social learning, as described by Hardy et al. (2022), or do they suggest that cognitive biases may emerge as a result of social learning?**

## **2d. Exploring an Additional Parameter in Belief Updating: Now, select one additional parameter (other than observations_per_generation and sample_size) that you believe is relevant to the cognitive aspects of non-social and social belief updating. Run the model using three different values for this parameter. How do these changes impact the outcomes? Specifically, from a "socio-cognitive" perspective: (i) Do your experiments provide an good illustration that social learning require less effort and fewer cognitive resources compared to non-social learning, as claimed by Hardy et al. (2022)? (ii) Do your experiments provide an good illustration of "rational information accumulation" through social learning, as described by Hardy et al. (2022), or do they suggest that cognitive biases may emerge as a result of social learning?**